# BData について

このノートブックでは Kamitani Lab の標準的な neural singal データフォーマットである BData について，その構造や簡単な使い方，簡単な作成手順を紹介します．

BData は bdpy (brain decoder toolbox for python) によって定義されるデータフォーマットで，主に preprocessing 済みの機械学習前 fMRI データのフォーマットとして使用されています．（過去には feature を格納していたケースもあります．） 
bdpy の各種関数を通して操作可能で，特定の stimulus と対応した sample の抽出，指定した ROI でのデータの取り出しなどを容易に行うことができます．一方で，メタデータが重くなりやすく，ファイル読み込みに時間がかかるといった課題も抱えています．


## BData の構造について

### Signal Data Structure

BData オブジェクトは，主に signal data を格納する `dataset` と，ROI や voxel の座標情報などを格納する `metadata` の 2 つのオブジェクトによって構成されています．

`dataset` は sample x voxel (+additional information) 次元の 2D matrix です．各行が機械学習で使用する 1 sample となっています．これは Kamitnai Lab だと基本的に fMRI 4 TR の平均データであることが多いです．
本来 3 次元の volume データである fMRI signal を flatten した状態で格納している点に注意してください．

まず，下記の操作を通して BData の `dataset` を確認してみましょう． `dataset` が 2 次元の matrix であり，float64 の数値データを格納していることが分かります．  


In [ ]:
# pip install は Colab 上でのみ実行してください．
# !pip install bdpy

import bdpy

print("Loading BData...")
bdata = bdpy.BData("data/fmri/test.h5")
print("Done.")
print()

print("bdata datashape:  {}".format(bdata.dataset.shape))
print("bdata datatype:   {}".format(bdata.dataset.dtype)) # bdata.dataSet でもアクセス可能
print("bdata values:", bdata.dataset[:8, :8])

一方，`metadata` は主に `key`, `value`, `description` の 3 つのコンポーネントによって構成されています．このうち，`value` が 2D matrix となっており，`dataset` と全く同じ列数を持ちます．これは `metadata.value` の各列と `dataset` の各列が，一対一で対応しているためです．

`metadata.value` は `dataset` と同様に，float64 の数値データの matrix になっています．

In [ ]:
print("Examples for bdata.metadata.key")
print(bdata.metadata.key[:10]) # bdata.metaData でもアクセス可能
print()
print("bdata.metadata.value")
print(bdata.metadata.value)
print()
print("Examples for bdata.metadata.description")
print(bdata.metadata.description[:10])
print()

print("bdata metadata.value datashape:  {}".format(bdata.metadata.value.shape))
print("bdata metadata.value datatype:   {}".format(bdata.metadata.value.dtype)) 


`dataset`と`metdata.value`の関係を図で表現すると下記のようになります．
`metadata.value` の ROI_V1 行が参照されると，その行において 1 の値を持つ列が `dataset` から抽出されます．これが ROI_V1 の signal data です．

なお，NaN 値が含まれていることもありますが，0 と NaN の列は抽出されません．

In [ ]:
from PIL import Image

img = Image.open("data/img/BData_structure.png")
img


`metadata.value` の行と対応するのが `metadata.key` および `metadata.description` の 2 つの list です．

`metadata.keylist` は `metadata.value` の各行が，どの ROI に対応しているのか，という情報を格納しています．

`metadata.description` はその ROI 情報の詳細／説明を格納しています．具体的には，元となった ROI mask ファイルのパスや，複数の ROI を連結した ROI の連結情報などを格納しています． 


In [ ]:
print("bdata metadata.key length:     {}".format(len(bdata.metadata.key)))
print("bdata metadata.key datatype:   {}".format(type(bdata.metadata.key))) 
print("bdata metadata.key:", bdata.metadata.key[:8])
print()

print("bdata metadata.description length:     {}".format(len(bdata.metadata.description)))
print("bdata metadata.description datatype:   {}".format(type(bdata.metadata.description))) 
print("bdata metadata.description:", bdata.metadata.description[:8])
print()



例えば下記の例では "ROI_V1" の signal data を，`BData.dataset` の 2D-matrix から抽出しています．これはまず "ROI_V1" という名前で `BData.metadata.key` を検索し，見つかった場合，その index を行番号とする `BData.metadata.value` の行を抽出します．そして，その行において $1$ の値を持つ列を `BData.dataset` から抽出します．


In [ ]:
import numpy as np

# Get ROI_V1 key index
key_index = bdata.metadata.key.index("ROI_V1")
print("ROI_V1 index in metadata.key:", key_index)

# Get ROI_V1 selector
vox_selector = bdata.metadata.value[key_index]
print("ROI_V1 selector:", vox_selector)
print("ROI_V1 voxel column index:", np.where(vox_selector == 1)[0])

# Extract ROI_V1 signal values
vox_selector = vox_selector.astype(bool)
vox = bdata.dataset[:, vox_selector]
print("ROI_V1 datashape:", vox.shape)


上記の操作を実際に行うのが BData の `get()` 関数です．通常はこちらを使用します．


In [ ]:
vox = bdata.get("ROI_V1")
print("ROI_V1 datashape:", vox.shape)


なお，BData の `metadata` を取得するための関数 `get_metadata()` や，`get()` をより高機能化させた `select()` を用いることで，上記の vox_selector を BData から直接取得することもできます．

`get_metadata()` は `metadata.value` から抽出した行のデータをそのまま返します．一方，`select()` の方はデータの形式 が bool にキャストされています．（列選択の直前の状態です．）これにより，`get_metadata()` で得られる値は np.nan 値を含むケースがあるので注意してください．


In [ ]:
print("Use get_metadata()")
vox_selector = bdata.get_metadata("ROI_V1")
print("  ROI_V1 selector:    {}".format(vox_selector))
print("  ROI_V1 voxel size:  {}".format(np.sum(vox_selector)))
print()

print("Use select() with `return_index` option")
vox, vox_selector = bdata.select("ROI_V1", return_index=True)
print("  ROI_V1 selector     {}".format(vox_selector))
print("  ROI_V1 voxel size:  {}".format(np.sum(vox_selector)))


まとめると，
- **`BData.dataset` へのアクセスには `get()` を用います．**
- **`BData.metadata` へのアクセスには，`get_metadata()` を用います．**
- **`dataset`と`metadata`への同時アクセスには `select()` を用います．**

`get_metadata()` の使用で最も多いケースは，voxel の座標値を取得するケースです．`bdpy.mri.create_bdata_fmriprep()` 関数を使用して作成された BData であれば（※1），`metadata` に `voxel_x`, `voxel_y`, `voxel_z`, `voxel_i`, `voxel_j`, `voxel_k` の行が格納されています．

`voxel_x`, `voxel_y`, `voxel_z` は対応する voxel の MNI 座標を示します．
`voxel_i`, `voxel_j`, `voxel_k` は対応する voxel の voxel 座標を示します．voxel 座標は 3次元 volume の端を原点とした matrix としての index のことです．

これらの座標情報を用いることで flatten された後の BData 内の signal 値を元の 3次元データに復元することも可能です．この詳細については別の資料 [BData to 3D to flattend](https://www.notion.so/BData-to-3D-to-flattend-1ad464d9d08d4b9d82add29c7e0d9522) をご確認ください．

※1 Kamitani Lab 標準の make_bdata.py はこの関数を使用して raw BData を作製しています．

In [ ]:
voxel_x, voxel_y, voxel_z = bdata.get_metadata("voxel_x"), bdata.get_metadata("voxel_y"), bdata.get_metadata("voxel_z")
print("voxel_x:", voxel_x)
print("voxel_y:", voxel_y)
print("voxel_z:", voxel_z)
print()

voxel_i, voxel_j, voxel_k = bdata.get_metadata("voxel_i"), bdata.get_metadata("voxel_j"), bdata.get_metadata("voxel_k")
print("voxel_i:", voxel_i)
print("voxel_j:", voxel_j)
print("voxel_k:", voxel_k)



また，これらの座標情報を上記の ROI の index 情報と組み合わせることで，特定の ROI，特定の voxel の座標情報などを取り出すこともできます．列方向を抽出する点に注意してください．


In [ ]:
voxel_x, voxel_y, voxel_z = bdata.get_metadata("voxel_x"), bdata.get_metadata("voxel_y"), bdata.get_metadata("voxel_z")
vox_selector = bdata.get_metadata("ROI_V1")
print("ROI_V1 voxel_x coordinate:", voxel_x[vox_selector.astype(bool)])
print("ROI_V1 voxel_y coordinate:", voxel_y[vox_selector.astype(bool)])
print("ROI_V1 voxel_z coordinate:", voxel_z[vox_selector.astype(bool)])


### Sample information

次に，voxel 以外のデータアクセスについて説明します．

`BData.dataset` には基本的に signal データが格納されていますが，他にも各 sample の刺激 index や run 番号，session 番号などが格納されています．

これらも `BData.metadata` を通じて，`BData.dataset` から取得します．例えば sample の run 番号を取り出したい場合は，下記のように `metadata.value` の `Run` 行が参照され，`dataset` から列ベクトルが取得されます．これが各 sample の run 番号を格納しています．


In [ ]:
from PIL import Image

img = Image.open("data/img/BData_structure2.png")
img


取得した Run index を用いて sample の選択を行いたい場合，flatten() する必要があることに注意してください．これは `bdata.get()` によって取り出された直後のデータが，`BData.dataset` から切り出された直後の形状，列ベクトルになっているためです．


In [ ]:
# Get run metadata
run_selector = bdata.get_metadata("Run")
print("run_selector:", run_selector)
print("run index:",    np.where(run_selector == 1)[0])
print()

# Get run index list
runs = bdata.get("Run")
print("runs.shape:", runs.shape)
print("runs:", runs) 
print()

# Error !!
# vox[runs == 1, :]

# Success
print("Run=1 voxel datashape:", vox[runs.flatten() == 1, :].shape)


まとめると，
- **`BData.dataset` の行，つまり sample に一対一対応する情報は，`BData.dataset` に格納されます．**
- **`BData.dataset` の列に対応する情報は `BData.metadata` に格納されます．**


### Label information

sample information の中で `Label` はやや特殊な扱いを受けます．このデータは他と同様に `get()` によって数値 index 情報として取得することもできますが，`BData.get_label()` を用いると数値データではなく，str 型の刺激名のlistとして取得することができます．


In [ ]:
print("数値 index として取得")
stimulus_index_array = bdata.get("stimulus_name")
print(stimulus_index_array)


In [ ]:
print("刺激名の list として取得")
stimulus_name_list = bdata.get_label("stimulus_name")
print(stimulus_name_list)

この刺激名は，BData  オブジェクト内にある vmap という辞書データを使って取得されています．これは `dataset` や `metadata` とはまた別の情報として BData 内に格納されています．

下記の `get_vmap()` に vmap テーブルを持つ列の key 名（ここでは "stimulus_name"）を指定すると，その数値 index と刺激名の対応辞書を確認できます．

なお，vmap は複数の列に付与したり，後から修正することも可能です．


In [ ]:
from pprint import pprint 

pprint(bdata.get_vmap("stimulus_name")) 

### BData の情報を一覧する

これまでは BData の構造理解のため，`dataset` や `metadata` に直接アクセスしていました．
しかし，基本的には　 `BData.dataset.shape` と `BData.show_metadata()` の 2 つで，その BData のおおよその情報を把握することが可能です．

In [ ]:
# BData の大きさを見る

bdata.dataset.shape

In [ ]:
# ROI の種類を見る

bdata.show_metadata()

### 補足: BData の予約語について

これまでの説明でもいくつか出てきましたが，BData の key にはいくつか予約語が存在しています．
ただし，Lab の `make_bdata.py` スクリプトによって作成された fMRI data の BData の場合に限ります．自分が 1 から BData object を作成した場合は，存在しません．


1. Voxel 座標の予約語
- `voxel_x`, `voxel_y`, `voxel_z`
    - 対応する voxel の MNI 座標
- `voxel_i`, `voxel_j`, `voxel_k`
    - 対応する voxel の voxel 座標


2. 各 sample の情報を示す予約語
- `Block`
- `Run`
- `Session`
- `Label`
- `MotionParameter`
- `MotionParameter_trans_x`, `MotionParameter_trans_y`, `MotionParameter_trans_z`, `MotionParameter_rot_x`, `MotionParameter_rot_y`, `MotionParameter_rot_z`
    - 基本的な Head motion 情報，MotionParameter の下位情報．
- `GlobalSignal`, `WhiteMatterSignal`, `CSFSignal`, `DVARS`, `STD_DVARS`, `FramewiseDisplacement`, `aCompCor`, `tCompCor`, `Cosine`
    - 追加の Head motion 情報．
- `trial_type`, `stimulus_name`, `response_time`
    - task event ファイルから BData に組み込まれた情報．したがって，task event ファイルでの列名を変更するとこれらも変更される．


3. Signal 範囲を示す予約後
- `VoxelData`
    - `BDData.dataset` のうち，voxel の列の範囲を示す（データによっては `VertexData` であることもある）



自分で BData を作成する場合，上記の key との重複に注意して作成してください．

## BData の作り方

ここでは BData の簡単な作り方を説明します．既存の fMRI データ（NifTi）から BData を作成する場合はこれとはまた別の手続きになります．ここではあくまで BData object の操作の基礎を学ぶために，単純なテストデータから BData を構築します．


### Step1: Create empty BDdata

まず，空の Bdata を作製してみましょう．

In [ ]:
bdata = bdpy.BData()
print(bdata.dataset.shape)


### Step2: VoxelData を追加

10 x 20 の random value matrix を作成します．これは 10 sample 20 voxel のデータを模擬しています．

これを `VoxelData` として BData に追加します．

In [ ]:
import numpy as np

random_vox = np.random.random([10, 20]) 
bdata.add(random_vox, name="VoxelData")

# Get added voxel data
vox = bdata.get("VoxelData")
print("VoxelData shape:", vox.shape)


### Step2: Run などの sample information を追加 

`VoxelData` に対して，sample information を追加します．ここでは Run index を付与してみましょう．この付与によって，`dataset` の列数が 1 つ増え，`metadata.key` に "Run" が追加されている様子が確認できます．


In [ ]:
print("Before add run index")
print("  BData dataset shape: {}".format(bdata.dataset.shape))
print("  BData metadata.key:  {}".format(bdata.metadata.key))

run_index = np.array([1,1,1,1,1,2,2,2,2,2]) # The first 5 samples are from the 1st run. The remaining samples are from the 2nd run.
run_index = run_index.reshape(-1, 1)
bdata.add(run_index, name="Run")

print("After add run index")
print("  BData dataset shape: {}".format(bdata.dataset.shape))
print("  BData metadata.key:  {}".format(bdata.metadata.key))

追加した列の情報を元に sample を選択します．


In [ ]:
runs = bdata.get("Run").flatten()
vox = bdata.get("VoxelData")[runs == 1]
print("1st Run's voxel shape:", vox.shape)

### Step3: Label を追加

刺激の情報を追加します．ここでは vmap も追加し，`get_label()` によって刺激名が得られるようにします．

In [ ]:
sample_index = np.array([1,2,3,4,5,6,7,8,9,10])
vmap = {
    1: "dog", 
    2: "cat",
    3: "chicken",
    4: "flamingo", 
    5: "snake",
    6: "lizard",
    7: "flog",
    8: "axolotl",
    9: "goldenfish",
    10: "shark",
}
bdata.add(sample_index, name="stimulus_name")
bdata.add_vmap(key="stimulus_name", vmap=vmap)

print(bdata.get("stimulus_name"))
print(bdata.get_label("stimulus_name"))


これにカテゴリ名の vmap を追加してみましょう．

In [ ]:
category_index = np.array([1,1,2,2,3,3,4,4,5,5])
vmap = {
    1: "mammal", 
    2: "bird",
    3: "reptile", # 爬虫類
    4: "amphibian", # 両生類
    5: "fish", 
}
bdata.add(category_index, name="category_name")
bdata.add_vmap(key="category_name", vmap=vmap)

print(bdata.get("category_name"))
print(bdata.get_label("category_name"))


例えば，哺乳類の 2 sample だけを抽出する場合は，以下のように操作します．

In [ ]:
vox = bdata.get("VoxelData")
categories = np.array(bdata.get_label("category_name")) # array に変換してください
mammal_vox = vox[categories == "mammal", :]
print(mammal_vox.shape)


### Step4: metadata (ROI) を追加

`VoxelData` の下に新たな ROI を追加します．ここでは 20 voxel のうち最初の 8 voxel が ROI_V1 に該当すると仮定します．

voxel_size と同じ大きさの boolean 配列を作成し，"ROI_V1" として指定する voxel の列だけ True にします．そして，`add_metadata()` を実行するときに `where` オプションを指定すると，ここで指定された範囲のみに対応して "ROI_V1" の列を指定するベクトルが `metadata.value` に追加されるようになります．


In [ ]:
# VoxelData の列数から voxel size を取得する
voxel_size = bdata.get("VoxelData").shape[1]
print("voxel size:            {}".format(voxel_size))
 
# ROI_V1 の範囲を指定する boolean 配列を作成する （最初の 8 voxel を ROI_V1 とする）
vox_selector = np.zeros(voxel_size).astype(bool) 
vox_selector[0:8] = True
print("V1 voxel in VoxelData: {}".format(vox_selector))

# VoxelData の下に ROI_V1 の ROI 情報を追加する
bdata.add_metadata(key="ROI_V1", value=vox_selector, where="VoxelData")

# ROI_V1 を取得する
vox, vox_selector = bdata.select("ROI_V1", return_index=True)
print("ROI_V1 dataset shape:  {}".format(vox.shape))


なお，`metadata.value` に追加されるとき，`where` の範囲外は NaN 値が入ります．今回は "Run"と"stimulus_name"，"category_name"の列をすでに追加しているため，最後の 3 列は nan 値となります．

`add_metadata()` 時に明示的に指定されていない部分は，NaN で埋められることに留意してください．


In [ ]:
vox_selector = bdata.get_metadata("ROI_V1")
print(vox_selector)


In [ ]:
print(bdata.metadata.value)

また，もう一つ重要な注意点として，実際に我々が既存の fMRI BData に ROI を追加する場合，上記よりも複雑な手続きが必要です．
まず，必要な ROI mask ファイルを作成し，それから `bdpy.mri.roi.add_roimask()` 関数を用います．この関数は `voxel_x`, `voxel_y`, `voxel_z` などの座標情報に基づいて追加すべき voxel 列の選択を行います．ここで紹介した手続は，簡易版であることをご認識ください．．


## 補足

### NifTI, BIDS 構造からの BData 作成

NifTI データからの BData 作成には主に下記の関数が関わっています．どちらも `bdpy.mri` の下にあります．

- `create_bdata_fmriprep()`
    - BIDS ディレクトリのパスを指定すると，そのディレクトリ下の複数の NifTI データを読み込み，1 つの BData ファイルまたは，BData のリストとして返す関数
- `BrainData()`
    - NifTI ファイルのパスを指定すると，その signal データや座標情報を抽出した object を生成する class
    
`make_bdata.py` からは `create_bdata_fmriprep()` が呼び出されて使用されています．


### BData の削除について

現状，bdpy では BData から列や key の削除を行う機能が実装されていません．
ただし，上記の通り，基本は行列操作によってこれらの削除は容易に行えます．

特に古い BData は非常にデータが重いことが多いため，データ共有や公開などを行う場合は，列や key を削除したものを用意する方が望ましいことが多いです．


In [ ]:
import numpy as np
import fnmatch


def remove_data(bdata, remover):
    """
    Remove selected ROIs (columns) from a BData object.

    Columns for which ``remover`` is True are removed from both
    ``bdata.dataset`` and ``bdata.metadata.value``. The input object
    is modified in-place and returned.

    Parameters
    ----------
    bdata : BData
    
    remover : array_like of bool, shape (n_roi,)
        One-dimensional boolean mask specifying which ROIs (columns)
        to remove. Columns with True are removed; columns with False
        are kept. The length must match the number of columns in
        ``bdata.dataset`` and ``bdata.metadata.value``.

    Returns
    -------
    bdata : BData object after column removal.
    
    Notes
    -----
    This function mutates ``bdata`` in-place. If you need to preserve
    the original object, create a copy beforehand.
    """
    bdata.dataset = bdata.dataset[:, np.logical_not(remover)]
    bdata.metadata.value = bdata.metadata.value[:, np.logical_not(remover)]
    return bdata
    
    
def remove_metadata(bdata, keypattern, verbose=True):
    """
    Remove metadata entries that match a key pattern.

    Uses :func:`fnmatch.fnmatchcase` for pattern matching and removes
    metadata rows (key, value, description) whose keys match the given
    pattern. Wildcards are supported, e.g. ``"hcp180_*"``. The input
    object is modified in-place and returned.

    Parameters
    ----------
    bdata : BData
    
    keypattern : str
        Pattern string used to select metadata keys to remove. Passed
        directly to :func:`fnmatch.fnmatchcase`. For example:
        ``"hcp180_*"``, ``"freesurfer_*"``, or ``"*motion*"``.
        
    verbose : bool, default True
        If True, prints the names of removed keys to standard output.
        Set to False to disable logging.

    Returns
    -------
    bdata : BData object after metadata removal.

    Notes
    -----
    This function mutates ``bdata`` in-place. If you need to preserve
    the original metadata, create a copy beforehand. 
    """
    # 対応する metadatakey を検出
    survive_keys = np.ones(bdata.metadata.value.shape[0]).astype(bool)
    for i, metakey in enumerate(bdata.metadata.key):
        if fnmatch.fnmatchcase(metakey, keypattern):
            survive_keys[i] = False
            if verbose:
                print("Remove:", metakey)
    # 削除
    key = list(np.array(bdata.metadata.key)[survive_keys])
    value = bdata.metadata.value[survive_keys]
    desc = list(np.array(bdata.metadata.description)[survive_keys])
    # 置換
    bdata.metadata.key = key
    bdata.metadata.value = value
    bdata.metadata.description = desc
    return bdata